[**Download Chapter pdf**](04-time_series.md.pdf)

[**Download Chapter notebook (ipynb)**](04-time_series.ipynb)

[<span style="color: rgb(255, 0, 0);">**Mandatory Lesson Feedback Survey**</span>](https://docs.google.com/forms/d/e/1FAIpQLSdr0capF7jloJhPH3Pki1B3LZoKOG16poOpuVJ7SL2LkwLHQA/viewform?pli=1)

- How is timeseries data visualised?
- Why do we need to tidy up/filter the data?
- How to study correlation among timeseries data points? 


- Learning ways to display multiple time series.
- Understanding why filtering is needed.
- Explaining fourier spectrum of time series.
- Knowledge of correlation matrix of time series.


## Prerequisites

- [Dataframes 1 and 2](01-data_frames_1.Rmd)
- [Image Handling](03-image_handling.Rmd)
- [Basics of Numpy Arrays]()



In [ ]:
from pandas import read_csv

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): ModuleNotFoundError: No module named 'pandas'

Detailed traceback:
  File "<string>", line 1, in <module>
  File "/home/runner/.local/share/renv/cache/v5/R-4.2/x86_64-pc-linux-gnu/reticulate/1.24/ffdf27627a3c1537478073c43b6e7980/reticulate/python/rpytools/loader.py", line 39, in _import_hook
    module = _import(
```

In [ ]:
from numpy import arange, zeros, linspace, sin, pi, c_, mean, var, array
from numpy import correlate, corrcoef, fill_diagonal, amin, amax, asarray
from numpy import around
from numpy.ma import masked_less, masked_greater

from matplotlib.pyplot import subplots, yticks, legend, axis, figure, show

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): ModuleNotFoundError: No module named 'matplotlib'

Detailed traceback:
  File "<string>", line 1, in <module>
  File "/home/runner/.local/share/renv/cache/v5/R-4.2/x86_64-pc-linux-gnu/reticulate/1.24/ffdf27627a3c1537478073c43b6e7980/reticulate/python/rpytools/loader.py", line 39, in _import_hook
    module = _import(
```

## Python Function

<p style='text-align: justify;'>
Please execute the following function definition before proceeding. The function code takes data and creates a plot of all columns as time series, one above the other. When you execute the function code nothing happens. Similar to the import, running a function code will only activate it and make it available for later use.
</p>


In [ ]:
def plot_series(data, sr):
    '''
    Time series plot of multiple time series
    Data are normalised to mean=0 and var=1 
    
    data: nxm numpy array. Rows are time points, columns are recordings
    sr: sampling rate, same time units as period
    '''

    samples = data.shape[0]
    sensors = data.shape[1]
    
    period = samples // sr

    time = linspace(0, period, period*sr)

    offset = 5 # for mean=0 and var=1 normalised data

    # Calculate means and standard deviations of all columns
    means = data.mean(axis=0)
    stds = data.std(axis=0)

    # Plot each series with an offset
    fig, ax = subplots(figsize=(7, 8))

    ax.plot(time, (data - means)/stds + offset*arange(sensors-1,-1,-1));

    ax.plot(time, zeros((samples, sensors)) + offset*arange(sensors-1,-1,-1),'--',color='gray');

    yticks([]);

    names = [str(x) for x in range(sensors)]
    legend(names)
        
    ax.set(xlabel='Time')

    axis('tight');
    
    return fig, ax

## Example: Normal and Pathological EEG

<p style='text-align: justify;'>
As an example, let us import two sets of time series data and convert them to numpy arrays, here called _data_back_ and _data_epil_. They represent human electroencephalogram (EEG) as recorded during normal _background_ activity and during an epileptic seizure called _absence_ seizure. 
</p>


In [ ]:
df_back = read_csv("data/EEG_background.txt", delim_whitespace=True) 

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'read_csv' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
df_epil = read_csv("data/EEG_absence.txt", delim_whitespace=True) 

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'read_csv' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
sr = 256     # 1 / seconds
period = 6   # seconds
channels = 10

d1 = df_back.to_numpy()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'df_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
d2 = df_epil.to_numpy()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'df_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
data_back = d1[:period*sr:, :channels]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'd1' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
data_epil = d2[:period*sr:, :channels]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'd2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

<p style='text-align: justify;'>
The `read_csv` function is used with the keyword argument `delim_whitespace`. When set to True, this allows to import data that are space-separated (rather than comma-separated). If you check the data .txt files, you will see that the numbers (which represent voltages) are indeed separated by space, not comma. 
</p>

<p style='text-align: justify;'>
Next, three constants are assigned: The sampling rate, sr, which is given in number of samples recorded per seconds; the duration of the recording, period, which is given in seconds; and the number of columns, referred to as channels, to be extracted from the recording. We use the first 10 columns in this lesson.
</p>

The data are then converted from a data frame to Numpy arrays.

To see the names of the channels (or recording sensors) we can use `head`. 


In [ ]:
df_back.head()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'df_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

<p style='text-align: justify;'>
The row indices and column names for the seizure data look the same. The names of the recording channels are from the commonly used [10-20 system](https://en.wikipedia.org/wiki/10–20_system_(EEG)) to record voltages of brain activity from the scalp in humans. E.g. 'F' stands for the frontal lobe.  
</p>

<p style='text-align: justify;'>
We can now use the plot function from above to plot the data. Note from the definition (first line) that two input arguments are reqired: the data set and the sampling rate. 
</p>


In [ ]:
plot_series(data_back, sr)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

In [ ]:
plot_series(data_epil, sr);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

__Observations__

1. Background:

- There are irregular oscillations of all recorded brain potentials.

- Oscillations recorded at different locations above the brain differ.

- Oscillations are not stable but modulated over time.

- There are different frequency components in each trace.

2. Epileptic Seizure:

- There are regular oscillations.

- Oscillations recorded at different locations are not identical but similar or at least related in shape.

- Despite some modulation, oscillations are fairly stable over time.

- There are repetitive motifs composed of two major components throughout the recording, a sharp _spike_ and a slow _wave_. 

__Task__

<p style='text-align: justify;'>
Quantify features of these time series data to get an overview. As a univariate feature we can use the frequency content. This takes into account the fact that the rows (or samplles) are not independent of each other but are organised along the time axis. In consequence, there are correlations between data points along the rows of each column and the __Fourier spectrum__ can be used to identify these. 
</p>

<p style='text-align: justify;'>
The Fourier spectrum assumes that the data are stationary and can be thought of as a superposition of regular sine waves with different frequencies. Its output will show which of the frequencies are present in the data and also their respective amplitudes. 
</p>

As a bivariate feature we can use the cross-correlation matrix.

## Work Through Example

Check the Numpy array containing the background and seizure data. 


In [ ]:
print(data_back.shape, data_epil.shape)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

There are 1536 rows and 10 columns.

### **Display data with offset**

<p style='text-align: justify;'>
Take a look at the code of the function `plot_series` that creates the time series plot. It requires the input of a data file where the row index is interpreted as time. In addition, the sampling rate sr is required to be able to extract the time scale. The sampling rate specifies the number of samples recorded per unit time. 
</p>

The sensors or recording channels are assumed to be in the columns. 


In [ ]:
def plot_series(data, sr):
    '''
    Time series plot of multiple time series
    Data are normalised to mean=0 and var=1 
    
    data: nxm numpy array. Rows are time points, columns are channels
    sr: sampling rate, same time units as period
    '''

    samples = data.shape[0]
    sensors = data.shape[1]
    
    period = samples // sr

    time = linspace(0, period, period*sr)

    offset = 5 # for mean=0 and var=1 normalised data

    # Calculate means and standard deviations of all columns
    means = data.mean(axis=0)
    stds = data.std(axis=0)

    # Plot each series with an offset
    fig, ax = subplots(figsize=(7, 8))

    ax.plot(time, (data - means)/stds + offset*arange(sensors-1,-1,-1));

    ax.plot(time, zeros((samples, sensors)) + offset*arange(sensors-1,-1,-1),'--',color='gray');

    yticks([]);

    names = [str(x) for x in range(sensors)]
    legend(names)
        
    ax.set(xlabel='Time')

    axis('tight');
   
    return fig, ax

The declaration syntax `def` is followed by the function name and, in parenthesis, the input arguments. It is completed with a colon.

Following the declaration is the documentation of the function. 

Next comes the function code, all indented. 

The function closes with the optional output syntax `return` and any number of returned variables, anything that might be used as a product of running the function. 

In our example, the figure environment and the coordinate system are 'returned', and can in principle be used to further modify the plot. 

Here is how to call the function and then add a title and the sensor names to the display.



In [ ]:
(fig, ax) = plot_series(data_epil, sr)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
names = df_back.columns[:channels]
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'df_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig.suptitle('Recording of Absence Seizure', fontsize=16);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
legend(names);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'legend' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

<p style='text-align: justify;'>
A function usually (but not necessarily) takes in one or several variables or values, processes them, and produces a specific result. The variable(s) given to a function and those produced by it are referred to as input arguments, and outputs respectively.
</p>
<p style='text-align: justify;'>
There are different ways to create functions in Python. In this course, we will be using `def` to implement our own functions. This is the easiest, and by far the most common method for declaring functions. The structure of a typical function defined using `def` can be see in the `plot_series` example:
</p>

There are several points to remember in relation to functions:

- The name of a function follows same principles as that of any other variable. It must be in lower-case characters.

- The input arguments of a function,  e.g. _data_ and _sr_ in the example, are essentially variables whose scope is the function. That is, they are only accessible within the function itself, and not from outside the function.

- Variables defined inside of a function, should not use the same name as variables defined outside. Otherwise they may override each other. 

<p style='text-align: justify;'>
It is important for a function to only perform one specific task. As such it can be used independent of the current context. Try to avoid incorporating separable tasks into a single function.
</p>
<p style='text-align: justify;'>
Once you start creating functions for different purposes you can start to build your own library of ready-to-use functions to address different needs. This is the primary principle of a popular programming paradigm known as [functional programming](https://en.wikipedia.org/wiki/Functional_programming).
</p>

### **Filtering**

Data sets with complex waveforms contain many different components which may or may not be relevant for a specific question. 
<p style='text-align: justify;'>
Data filtering is applied to take out specific components. Component in this context refers to 'frequency', i.e.  the number of cycles per unit of time. Thus a small number refers to low frequencies with long periods (cycles) and a large number to high frequencies with short periods. 
</p>

Let us see a simple example how both low- and high-frequency components can be filtered (suppressed) in the example time series. 

Here is a simple function which takes two additional input arguments, the low and the high cut-off.



In [ ]:
def data_filter(data, sr, low, high):
    """
    Filtering of multiple time series. 

    data: nxm numpy array. Rows are time points, columns are recordings
    sr: sampling rate, same time units as period    
    
    low:  Low cut-off frequency (high-pass filter)
    high: High cut-off frequency (low-pass filter)

    return: filtered data
    """

    from scipy.signal import butter, sosfilt

    order = 5

    filter_settings = [low, high, order]

    sos = butter(order, (low,high), btype='bandpass', fs=sr, output='sos')

    data_filtered = zeros((data.shape[0], data.shape[1]))

    for index, column in enumerate(data.transpose()): 
        forward = sosfilt(sos, column)
        backwards = sosfilt(sos, forward[-1::-1])
        data_filtered[:, index] = backwards[-1::-1]
        
    return data_filtered
    

In [ ]:
data_back_filt = data_filter(data_back, sr, 8, 13)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
(fig, ax) = plot_series(data_back_filt, sr)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_back_filt' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig.suptitle('Filtered Recording of Background EEG', fontsize=16);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
legend(names);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'legend' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

<p style='text-align: justify;'>
The frequency range from 8 to 13 Hz is referred to as alpha band in the electroencephalogram. It is thought that this represents a kind of idling rhythm of the brain, i.e. an activity where the brain is not activily processing sensory input. 
</p>

## __DIY1: Band-pass filtered data__

Create figures of the delta (1-4 Hz) band for both the background and the seizure EEG. Note the differences.

## DIY ANSWER


In [ ]:
data_back_filt = data_filter(data_back, sr, 1, 4)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
(fig, ax) = plot_series(data_back_filt, sr)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_back_filt' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig.suptitle('Delta Band of Background EEG', fontsize=16);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
legend(names);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'legend' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

In [ ]:
data_epil_filt = data_filter(data_epil, sr, 1, 4)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
(fig, ax) = plot_series(data_epil_filt, sr)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_epil_filt' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig.suptitle('Delta Band of Seizure EEG', fontsize=16);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
legend(names);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'legend' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

### **Fourier Spectrum**
<p style='text-align: justify;'>
The Fourier spectrum decomposes the time series into a sum of sine waves. The spectrum gives the coefficients of each of the sine wave components. The coefficients are directly related to the amplitudes needed to optimally fit the sum of all sine waves to recreate the original data. 
</p>

<p style='text-align: justify;'>
However, the assumption behind the Fourier transform is that the data are provided as in infinitely long stationary time series. These assumptions are not fullfilled as the data are finite and stationarity of a biological system can typically not be guaranteed. Thus, interpretation needs to be cautious. 
</p>

#### **Fourier Transform of EEG data**

We import the Fourier transform function `fft` from __scipy.fftpack__ and can use it to transform all columns at the same time. 



In [ ]:
from scipy.fftpack import fft

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): ModuleNotFoundError: No module named 'scipy'

Detailed traceback:
  File "<string>", line 1, in <module>
  File "/home/runner/.local/share/renv/cache/v5/R-4.2/x86_64-pc-linux-gnu/reticulate/1.24/ffdf27627a3c1537478073c43b6e7980/reticulate/python/rpytools/loader.py", line 39, in _import_hook
    module = _import(
```

In [ ]:
data_back_fft = fft(data_back, axis=0)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fft' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

To plot the results, a couple of steps are required. 
<p style='text-align: justify;'>
First, we obtain a Fourier spectrum for every data column, so we need to define how many plots we want to have. If we take only columns, we can display them all in one go.
</p>
<p style='text-align: justify;'>
Second, the Fourier transform results in twice the number of complex coefficients (positive and negative) of which we only need the first half. 
</p>
<p style='text-align: justify;'>
Third, the Fourier transform outputs complex numbers. To display the 'amplitude' of a frequency (the coefficient corresponding to the amplitude of the sine wave with that frequency) we take the absolute value of the complex numbers. 
</p>


In [ ]:
no_win = 2

rows = data_back.shape[0]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
freqs = (sr/2)*linspace(0, 1, int(rows/2))

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'rows' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
amplitudes_back = (2.0 / rows) * abs(data_back_fft[:rows//2, :2])


```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'rows' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fig, axes = subplots(figsize=(6, 5), ncols=1, nrows=no_win, sharex=False)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
names = df_back.columns[:2]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'df_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
for index, ax in enumerate(axes.flat):
    axes[index].plot(freqs, amplitudes_back[:, index])
    axes[index].set_xlim(0, 8)
    axes[index].set(ylabel=f'Amplitude {names[index]}')

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
axes[index].set(xlabel='Frequency (Hz)');

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
show()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

We can see that in these two channels, the main amplitude contributions lie in the low frequencies, below 2 Hz. 

Let us compare the corresponding figure for the case of seizure activity:



In [ ]:
data_epil_fft = fft(data_epil, axis=0)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fft' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```


In [ ]:
fig, axes = subplots(figsize=(6, 5), ncols=1, nrows=no_win, sharex=False)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
names = df_epil.columns[:2]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'df_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
amplitudes_epil = (2.0 / rows) * abs(data_epil_fft[:rows//2, :2])

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'rows' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
for index, ax in enumerate(axes.flat):
    axes[index].plot(freqs, amplitudes_epil[:, index])
    axes[index].set_xlim(0, 12)
    axes[index].set(ylabel=f'Amplitude {names[index]}')

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
axes[index].set(xlabel='Frequency (Hz)');

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
show()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

The main frequency of the epileptic rhythm is between 2 and 3 Hz. 
<p style='text-align: justify;'>
As we can see above in the Fourier spectra above, the amplitudes are high for low frequencies and tend to decrease with increasing frequency. Sometimes it is useful to see the high frequencies enhanced. This can be achieved with a logarithmic plot of the powers.
</p>


In [ ]:
fig, axes = subplots(figsize=(6, 6), ncols=1, nrows=no_win, sharex=False)
                    

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
for index, ax in enumerate(axes.flat):

    axes[index].plot(freqs, amplitudes_back[:, index])
    axes[index].set_xlim(0, 30)
    axes[index].set(ylabel=f'Amplitude {names[index]}')
    axes[index].set_yscale('log')

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
axes[no_win-1].set(xlabel='Frequency (Hz)');

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fig.suptitle('Logarithmic Fourier Spectra of Background EEG', fontsize=16);

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
show()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

And for the seizure data:


In [ ]:

fig, axes = subplots(figsize=(6, 10), ncols=1, nrows=no_win, sharex=False)
                    

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
for index, ax in enumerate(axes.flat):

    axes[index].plot(freqs, amplitudes_epil[:, index])
    axes[index].set_xlim(0, 30)
    axes[index].set(ylabel=f'Power {names[index]}')
    axes[index].set_yscale('log')

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
axes[no_win-1].set(xlabel='Frequency (Hz)');

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fig.suptitle('Logarithmic Fourier Spectra of Seizure EEG', fontsize=16);

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
show()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```
<p style='text-align: justify;'>
In the spectrum of the absence data, it is now more obvious that there are further maxima at 6, 9, 12, and perhaps 15Hz. These are integer multiples or 'harmonics' of the basic frequency at around 3Hz, also referred to as the fundamental frequency.
</p>
<p style='text-align: justify;'>
A feature that can be used as a summary statistic, is to caclulate the __band power__ for each channel. The band power can be obtained as the sum of all powers within a specified range of frequencies, also called the 'band'. The band power thus represents a single number. 
</p>

## __DIY2: Fourier spectra of filtered data__

Calculate and display the Fourier spectra of the first two channels filtered between 4 and 12 Hz for the absence seizure data. Can you find harmonics?
	

In [ ]:
data_epil_filt = data_filter(data_epil, sr, 4, 12)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
data_epil_fft = fft(data_epil_filt, axis=0)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fft' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
rows = data_epil.shape[0]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
freqs = (sr/2)*linspace(0, 1, int(rows/2))

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'rows' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
amplitudes_epil = (2.0 / rows) * abs(data_epil_fft[:rows//2, :no_win])

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'rows' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fig, axes = subplots(figsize=(6, 10), ncols=1, nrows=no_win, sharex=False)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
for index, ax in enumerate(axes.flat):
    axes[index].plot(freqs, amplitudes_epil[:, index])
    axes[index].set_xlim(0, 12)
    axes[index].set(ylabel=f'Amplitudes {names[index]}')

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
axes[no_win-1].set(xlabel='Frequency (Hz)');

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fig.suptitle('Fourier Spectra of Seizure EEG', fontsize=16);

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
show()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```


### **Cross-Correlation Matrix**
As one example of a multivariate analysis of time series data, we can calculate the cross-correlation matrix. 

Here it is done for the background:


In [ ]:
corr_matrix_back = corrcoef(data_back, rowvar=False)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fill_diagonal(corr_matrix_back, 0)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_matrix_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fig, ax = subplots(figsize = (8,8))

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
im = ax.imshow(corr_matrix_back, cmap='coolwarm');

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fig.colorbar(im, orientation='horizontal', shrink=0.68);

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
show()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

<p style='text-align: justify;'>
The diagonal is set to zero. This is done to improve the visual display. If left to one, the diagonal tends to dominate the visual impression even though it is trivial and nothing can be learned from it.
</p>

Looking at the non-diagonal elements, we find:

- two strongly correlated series (indices 5 and 7)

- two strongly anti-correlated series (indices 3 and 4)

- a block of pronounced correlations between series with indices 4 through 9)

## __DIY3: Display the correlation matrix for the seizure data__

Calculate the correlation matrix for the seizure data and compare the correlation pattern to the one from the background data.

## DIY ANSWER


In [ ]:
corr_matrix_epil = corrcoef(data_epil, rowvar=False)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fill_diagonal(corr_matrix_epil, 0)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_matrix_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fig, ax = subplots(figsize = (8,8))

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
im = ax.imshow(corr_matrix_epil, cmap='coolwarm');

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
fig.colorbar(im, orientation='horizontal', shrink=0.68);

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
show()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

<p style='text-align: justify;'>
We find - a number of pairs of strongly correlated series - two strongly anti-correlated series (in- dices 3 and 4) - a block of pronounced correlations between series with indices 4 through 9).
</p>
<p style='text-align: justify;'>
So interestingly, while the time series changes dramatically in shape, the correlation pattern still shows some qualitative resemblance.
</p>


<p style='text-align: justify;'> 
All results shown so far, represent the recording of the segment of 6 seconds chosen at the beginning. The human brain produces time-dependent voltage changes 24 hours a day and as seeing only a few seconds is only a partial view. The next step to investigate is to show how the features found for one segment vary over time. 
</p>

## Exercises

#### End of chapter Exercises

**Pathological Human Brain Rhythms**
<p style='text-align: justify;'>
Look at the image of brain activity from a child at the start of an epileptic seizure. It shows 4 seconds of evolution of the first 10 channels of a seizure rhythm at sampling rate sr=1024.
</p>



In [ ]:
path = 'data/P1_Seizure1.csv'

data = read_csv(path, delimiter=r"\s+")

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'read_csv' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
data_P1 = data.to_numpy()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
sr = 1024

period = 4

channels = 10

plot_series(data_P1[:sr*period, :channels], sr);

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_P1' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
show()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

Using the code from this lesson, import the data from the file `P1_Seizure1.csv` and generate an overview of uni- and multivariate features in the following form:

1. Pick the first two seconds of the recording as background, the last two second as epileptic seizure rhythm. Use the first ten channels in both cases. Data should have the shape (2048, 10).

2. Filter the data to get rid of frequencies below 1 Hz and frequencies faster than 20 Hz. 

3. Plot time series of both.

4. Fourier transform both filtered data sets and display the Fourier spectra of the first 4 channels. What are the strongest frequencies in the two sets?

5. Plot the correlation matrices of both data sets. Which challes show the strongest change in correlations?

## Please check these solutions only after submitting the assignments.


In [ ]:
from pandas import read_csv

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): ModuleNotFoundError: No module named 'pandas'

Detailed traceback:
  File "<string>", line 1, in <module>
  File "/home/runner/.local/share/renv/cache/v5/R-4.2/x86_64-pc-linux-gnu/reticulate/1.24/ffdf27627a3c1537478073c43b6e7980/reticulate/python/rpytools/loader.py", line 39, in _import_hook
    module = _import(
```

In [ ]:
from numpy import arange, zeros, linspace, sin, pi, c_, mean, var, array

from numpy import correlate, corrcoef, fill_diagonal, amin, amax, asarray

from numpy import around, triu_indices

from numpy.ma import masked_less, masked_greater

from scipy.fftpack import fft

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): ModuleNotFoundError: No module named 'scipy'

Detailed traceback:
  File "<string>", line 1, in <module>
  File "/home/runner/.local/share/renv/cache/v5/R-4.2/x86_64-pc-linux-gnu/reticulate/1.24/ffdf27627a3c1537478073c43b6e7980/reticulate/python/rpytools/loader.py", line 39, in _import_hook
    module = _import(
```

In [ ]:
from matplotlib.pyplot import subplots, yticks, legend, axis, figure, show

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): ModuleNotFoundError: No module named 'matplotlib'

Detailed traceback:
  File "<string>", line 1, in <module>
  File "/home/runner/.local/share/renv/cache/v5/R-4.2/x86_64-pc-linux-gnu/reticulate/1.24/ffdf27627a3c1537478073c43b6e7980/reticulate/python/rpytools/loader.py", line 39, in _import_hook
    module = _import(
```
### Q1
### Extract data for first and last two seconds

In [ ]:
sr = 1024

duration = 2 # seconds
data_n = data.to_numpy()

# dat_back = data.iloc[:duration*sr, :10]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
dat_back = data_n[:duration*sr, :10]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_n' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
dat_epil = data_n[data_n.shape[0] - duration*sr:, :10]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'data_n' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
time = linspace(0, 1, duration*sr)

print(dat_back.shape, dat_epil.shape, time.shape)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'dat_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

### Q2 and Q3
### Filter and display data


In [ ]:
def data_filter(data, sr, low, high):
    """
    Filtering of multiple time series. 

    data: nxm numpy array. Rows are time points, columns are recordings
    sr: sampling rate, same time units as period    
    
    low:  Low cut-off frequency (high-pass filter)
    high: High cut-off frequency (low-pass filter)

    return: filtered data
    """

    from scipy.signal import butter, sosfilt

    order = 5

    filter_settings = [low, high, order]

    sos = butter(order, (low,high), btype='bandpass', fs=sr, output='sos')

    data_filtered = zeros((data.shape[0], data.shape[1]))

    for index, column in enumerate(data.transpose()): 
        forward = sosfilt(sos, column)
        backwards = sosfilt(sos, forward[-1::-1])
        data_filtered[:, index] = backwards[-1::-1]
        
    return data_filtered

In [ ]:

dat_back_filt = data_filter(dat_back, sr, 1, 20)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'dat_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
(fig, ax) = plot_series(dat_back_filt, sr)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'dat_back_filt' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig.suptitle('First two seconds: Background EEG', fontsize=16);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

In [ ]:
dat_epil_filt = data_filter(dat_epil, sr, 1, 20)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'dat_epil' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
(fig, ax) = plot_series(dat_epil_filt, sr)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'dat_epil_filt' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig.suptitle('Last 2 seconds: Seizure EEG', fontsize=16);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

### Q4
### Fourier Transform


In [ ]:
rows = dat_back.shape[0]

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'dat_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
dat_back_fft = fft(dat_back_filt, axis=0)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fft' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
powers_back2 = (2.0 / rows) * abs(dat_back_fft[:rows//2, :])

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'rows' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
dat_epil_fft = fft(dat_epil_filt, axis=0)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fft' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
powers_epil2 = (2.0 / rows) * abs(dat_epil_fft[:rows//2, :])

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'rows' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

<p style='text-align: justify;'>
You can see the frequencies with largest power visually from a display of the Fourier spectrum. Here is an example code how to extract those values for each channel using Numpy functions.
</p>


In [ ]:
# Get the maxima of powers for each channel

powermax_back2 = amax(powers_back2, axis=0)
powermax_epil2 = amax(powers_epil2, axis=0)


# Get the frequency index of that maximum

powermax_back2_index = zeros(powers_back2.shape[1])

for ind, pow in enumerate(list(powers_back2.transpose())):
    pow_ind = list(pow).index(powermax_back2[ind])
    powermax_back2_index[ind] = pow_ind
    
powermax_epil2_index = zeros(powers_epil2.shape[1])

for ind, pow in enumerate(list(powers_epil2.transpose())):
    pow_ind = list(pow).index(powermax_epil2[ind])
    powermax_epil2_index[ind] = pow_ind
    
    
    
powermax_back2_freq = asarray(powermax_back2_index)*(sr / 2 / powers_back2.shape[0])
powermax_epil2_freq = asarray(powermax_epil2_index)*(sr / 2 / powers_epil2.shape[0])

print('Frequencies of max power in background (Hz): ', '\n', around(powermax_back2_freq, decimals=1))
print('')
print('Frequencies of max power in seizure (Hz): ', '\n', around(powermax_epil2_freq, decimals=1))    
    
#print(powermax_back2_freq, '\n',  powermax_epil2_freq)

fig, ax = subplots(nrows=2, figsize=(6,9))

binwidth = 1

(counts1, bins1, bars1) = ax[0].hist(powermax_back2_freq, bins=arange(0, 50 + binwidth, binwidth))
ax[0].set_xlim(0, 30)
ax[0].set_ylim(0, 10)
ax[0].set(xlabel='Frequency (Hz), Background')

(counts2, bins2, bars2) = ax[1].hist(powermax_epil2_freq, bins=arange(0, 50 + binwidth, binwidth))
ax[1].set_xlim(0, 30)
ax[1].set_ylim(0, 10)
ax[1].set(xlabel='Frequency (Hz), Seizure');

show()

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powers_back2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powers_epil2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powers_back2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powers_back2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powers_epil2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powers_epil2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powermax_back2_index' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powermax_epil2_index' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powermax_back2_freq' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'powermax_epil2_freq' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

### Fourier spectrum with max frequency


In [ ]:
print('Maximum count: background: ', amax(counts1))

print('Maximum count: seizure:    ', amax(counts2))

print('Frequency with maximum count: background: ', '10-11 Hz')

print('Frequency with maximum count: seizure:    ', ' 7- 8 Hz')

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'counts1' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'counts2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.output}
Frequency with maximum count: background:  10-11 Hz
Frequency with maximum count: seizure:      7- 8 Hz
```



In [ ]:
freqs = (sr/2)*linspace(0, 1, int(rows/2))
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'rows' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig, axes = subplots(figsize=(6, 14), ncols=1, nrows=4, sharex=False)
    
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[0].plot(freqs, powers_back2[:, 3])
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[0].set_xlim(0, 20)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[0].set_ylim(0, 12)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[0].set(ylabel=f'Power channel 3')
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[0].set(xlabel='Frequency (Hz)');
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[1].plot(time, dat_back_filt[:, 3], c='r')
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[1].set(xlabel='Time (s)');
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[1].set_ylim(-50, 60)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[2].plot(freqs, powers_epil2[:, 3])
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[2].set_xlim(0, 20)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[2].set_ylim(0, 12)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[2].set(ylabel='Power channel index 1')
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[2].set(xlabel='Frequency (Hz)');
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[3].plot(time, dat_epil_filt[:, 3], c='r')
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[3].set(xlabel='Time (s)');
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
axes[3].set_ylim(-50, 60)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'axes' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

### Q5
### Correlation Matrix


In [ ]:
corr_matrix_back2 = corrcoef(dat_back_filt, rowvar=False)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'dat_back_filt' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fill_diagonal(corr_matrix_back2, 0)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_matrix_back2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
corr_matrix_epil2 = corrcoef(dat_epil_filt, rowvar=False)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'dat_epil_filt' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fill_diagonal(corr_matrix_epil2, 0)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_matrix_epil2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig, ax = subplots(figsize = (8,8), ncols=2)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
im1 = ax[0].imshow(corr_matrix_back2, cmap='coolwarm');
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig.colorbar(im1, ax=ax[0], orientation='horizontal', shrink=0.8)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
im2 = ax[1].imshow(corr_matrix_epil2, cmap='coolwarm');
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig.colorbar(im2, ax=ax[1], orientation='horizontal', shrink=0.8);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'fig' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

### Hist of Correlation Coefficients


In [ ]:
channels = dat_back.shape[1]
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'dat_back' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
corr_coeffs_back2 = corr_matrix_back2[triu_indices(channels, k=1)]
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_matrix_back2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
corr_coeffs_epil2 = corr_matrix_epil2[triu_indices(channels, k=1)]
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_matrix_epil2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig, ax = subplots(nrows=2)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[0].hist(corr_coeffs_back2, bins = 12);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[0].set_xlim(-1, 1)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[1].hist(corr_coeffs_epil2, bins = 12);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[1].set_xlim(-1, 1);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

### Channel Correlations


In [ ]:
corr_coeffs_back2_mean = mean(abs(corr_matrix_back2), axis=0)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_matrix_back2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
corr_coeffs_epil2_mean = mean(abs(corr_matrix_epil2), axis=0)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_matrix_epil2' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
fig, ax = subplots(nrows=2, figsize=(5,9))
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
bins = arange(corr_coeffs_back2_mean.shape[0])
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_coeffs_back2_mean' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[0].bar(bins, corr_coeffs_back2_mean);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[0].set_xlabel('Background')
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[0].set_ylim(0, 0.4)
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[1].bar(bins, corr_coeffs_epil2_mean);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[1].set_xlabel('Seizure')
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax[1].set_ylim(0, 0.4);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

In [ ]:
corr_back2_mean = mean(corr_coeffs_back2_mean)
corr_epil2_mean = mean(corr_coeffs_epil2_mean)

print('Average correlation background: ', around(corr_back2_mean, decimals=2))
print('Average correlation seizure:    ', around(corr_epil2_mean, decimals=2))

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_coeffs_back2_mean' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_coeffs_epil2_mean' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_back2_mean' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_epil2_mean' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

### Channels with strongest correlations during the seizure

In [ ]:
threshold = 0.2

corr_coeffs_epil2_large = corr_coeffs_epil2_mean > threshold

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_coeffs_epil2_mean' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

In [ ]:
corr_coeffs_epil2_large

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_coeffs_epil2_large' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```

A horizontal line can be used to indicate the threshold:


In [ ]:
fig, ax = subplots()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'subplots' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
bins = arange(corr_coeffs_back2_mean.shape[0])
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_coeffs_back2_mean' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax.bar(bins, corr_coeffs_epil2_mean);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax.set_xlabel('Seizure')
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax.set_ylim(0, 0.4);
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
ax.axhline(y=threshold, c='r');
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ax' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
show()
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'show' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>

To find the indices of the channels that are larger than the threshold (i.e. those displaying True), we can use Numpy function `nonzero`: 


In [ ]:
from numpy import nonzero

nonzero(corr_coeffs_epil2_large)

```{.error}
Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'corr_coeffs_epil2_large' is not defined

Detailed traceback:
  File "<string>", line 1, in <module>
```



- `plot_series` is a Python function created to display multiple timeseries plots.
- Data filtering is applied to take out specific and relevant components.  
- The Fourier spectrum decomposes the time series into a sum of sine waves. 
- Cross-correlation matrix is used for multivariate analysis.
